In [6]:
import json
import tqdm
import nltk

import ssl 
ssl._create_default_https_context = ssl._create_unverified_context


nltk.download('punkt')
from collections import Counter
from tqdm import tqdm
import re
from squad.utils import get_word_span, get_word_idx, process_tokens

sent_tokenize = nltk.sent_tokenize
def word_tokenize(tokens):
    return [token.replace("''", '"').replace("``", '"') for token in nltk.word_tokenize(tokens)]

source_path='dev-v1.1.json'
source_data=json.load(open(source_path,'r'))

q, cq, y, rx, rcx, ids, idxs = [], [], [], [], [], [], []
na = []
cy = []
x, cx = [], []
answerss = []
p = []
word_counter, char_counter, lower_word_counter = Counter(), Counter(), Counter()
for ai, article in enumerate(tqdm(source_data['data'])):
    xp, cxp = [], []
    pp = []
    x.append(xp)
    cx.append(cxp)
    p.append(pp)
    for pi, para in enumerate(article['paragraphs']):
        # wordss
        context = para['context']
        context = context.replace("''", '" ')
        context = context.replace("``", '" ')
        
        xi = list(map(word_tokenize, sent_tokenize(context)))
        xi = [process_tokens(tokens) for tokens in xi]  # process tokens
        # given xi, add chars
        cxi = [[list(xijk) for xijk in xij] for xij in xi]
        xp.append(xi)
        cxp.append(cxi)
        pp.append(context)

        for xij in xi:
            for xijk in xij:
                word_counter[xijk] += len(para['qas'])
                lower_word_counter[xijk.lower()] += len(para['qas'])
                for xijkl in xijk:
                    char_counter[xijkl] += len(para['qas'])

        rxi = [ai, pi]
        assert len(x) - 1 == ai
        assert len(x[ai]) - 1 == pi
        for qa in para['qas']:
            # get words
            qi = word_tokenize(qa['question'])
            qi = process_tokens(qi)
            cqi = [list(qij) for qij in qi]
            yi = []
            cyi = []
            answers = []
            for answer in qa['answers']:
                answer_text = answer['text']
                answers.append(answer_text)
                answer_start = answer['answer_start']
                answer_stop = answer_start + len(answer_text)
                # TODO : put some function that gives word_start, word_stop here
                yi0, yi1 = get_word_span(context, xi, answer_start, answer_stop)
                # yi0 = answer['answer_word_start'] or [0, 0]
                # yi1 = answer['answer_word_stop'] or [0, 1]
                assert len(xi[yi0[0]]) > yi0[1]
                assert len(xi[yi1[0]]) >= yi1[1]
                w0 = xi[yi0[0]][yi0[1]]
                w1 = xi[yi1[0]][yi1[1] - 1]
                i0 = get_word_idx(context, xi, yi0)
                i1 = get_word_idx(context, xi, (yi1[0], yi1[1] - 1))
                cyi0 = answer_start - i0
                cyi1 = answer_stop - i1 - 1
                # print(answer_text, w0[cyi0:], w1[:cyi1+1])
                assert answer_text[0] == w0[cyi0], (answer_text, w0, cyi0)
                assert answer_text[-1] == w1[cyi1]
                assert cyi0 < 32, (answer_text, w0)
                assert cyi1 < 32, (answer_text, w1)

                yi.append([yi0, yi1])
                cyi.append([cyi0, cyi1])

            if len(qa['answers']) == 0:
                yi.append([(0, 0), (0, 1)])
                cyi.append([0, 1])
                na.append(True)
            else:
                na.append(False)

            for qij in qi:
                word_counter[qij] += 1
                lower_word_counter[qij.lower()] += 1
                for qijk in qij:
                    char_counter[qijk] += 1

            q.append(qi)
            cq.append(cqi)
            y.append(yi)
            cy.append(cyi)
            rx.append(rxi)
            rcx.append(rxi)
            ids.append(qa['id'])
            idxs.append(len(idxs))
            answerss.append(answers)

  0%|          | 0/48 [00:00<?, ?it/s]

[nltk_data] Downloading package punkt to /Users/xiongfei/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


  2%|▏         | 1/48 [00:01<00:57,  1.22s/it]

  4%|▍         | 2/48 [00:01<00:46,  1.00s/it]

  6%|▋         | 3/48 [00:01<00:34,  1.30it/s]

...

 92%|█████████▏| 44/48 [00:19<00:01,  2.55it/s]

 94%|█████████▍| 45/48 [00:19<00:01,  2.23it/s]

 96%|█████████▌| 46/48 [00:20<00:00,  2.24it/s]

 98%|█████████▊| 47/48 [00:20<00:00,  2.24it/s]

100%|██████████| 48/48 [00:21<00:00,  2.11it/s]

In [37]:
answerss

[['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],
 ['Carolina Panthers', 'Carolina Panthers', 'Carolina Panthers'],
 ['Santa Clara, California',
  "Levi's Stadium",
  "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."],
 ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],
 ['gold', 'gold', 'gold'],
 ['"golden anniversary"', 'gold-themed', '"golden anniversary'],
 ['February 7, 2016', 'February 7', 'February 7, 2016'],
 ['American Football Conference',
  'American Football Conference',
  'American Football Conference'],
 ['"golden anniversary"', 'gold-themed', 'gold'],
 ['American Football Conference',
  'American Football Conference',
  'American Football Conference'],
 ['February 7, 2016', 'February 7', 'February 7, 2016'],
 ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],
 ["Levi's Stadium",
  "Levi's Stadium",
  "Levi's Stadium in the San Francisco Bay Area at Santa Clara"],
 ['Santa Clara', 'Santa Clara', 'Santa Clara'],
 ['Super Bowl L', '